In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [4]:
# Load data
containers_df = pd.read_csv("containers.csv")
items_df = pd.read_csv("input_items.csv")

In [6]:
# Compute volume for both-
containers_df["container_volume"] = containers_df["height_cm"] * containers_df["depth_cm"] * containers_df["width_cm"]
containers_df["available_volume"] = containers_df["container_volume"]
items_df["item_volume"] = items_df["height_cm"] * items_df["depth_cm"] * items_df["width_cm"]

In [8]:
# Generate training data: simulate placement results
training_data = []
for _, item in items_df.iterrows():
    for _, container in containers_df.iterrows():
        fits = 1 if item["item_volume"] <= container["available_volume"] else 0
        training_data.append({
            "item_volume": item["item_volume"],
            "item_mass": item["mass_kg"],
            "priority": item["priority"],
            "container_volume": container["container_volume"],
            "same_zone": int(item["preferred_zone"] == container["zone"]),
            "fits": fits
        })

df_train = pd.DataFrame(training_data)

In [14]:
# Train/Test Split
X = df_train.drop("fits", axis=1)
y = df_train["fits"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
# Train ML model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Now predict placement using model
items_df["placement"] = None
items_df["placement_note"] = None

In [22]:
for iidx, item in items_df.iterrows():
    item_volume = item["item_volume"]
    item_mass = item["mass_kg"]
    priority = item["priority"]
    preferred_zone = item["preferred_zone"]

    scored_containers = []
    for cidx, container in containers_df.iterrows():
        feature = pd.DataFrame([{
            "item_volume": item_volume,
            "item_mass": item_mass,
            "priority": priority,
            "container_volume": container["container_volume"],
            "same_zone": int(preferred_zone == container["zone"])
        }])

        proba = model.predict_proba(feature)[0]
if len(proba) > 1:
    prediction = proba[1]
else:
    prediction = proba[0]  # or 0.0 if you're expecting class=1
  # probability of fit
    scored_containers.append((prediction, container["container_id"], cidx, container["available_volume"]))

    # Sort by highest prediction + available space
    scored_containers.sort(reverse=True)

    placed = False
    for score, cid, cidx, avail_vol in scored_containers:
        if avail_vol >= item_volume:
            containers_df.at[cidx, "available_volume"] -= item_volume
            items_df.at[iidx, "placement"] = cid
            note = "ML-predicted placement (preferred)" if preferred_zone == containers_df.at[cidx, "zone"] else "ML-predicted placement (alternate)"
            items_df.at[iidx, "placement_note"] = note
            placed = True
            break

    if not placed:
        items_df.at[iidx, "placement_note"] = "ML model could not find fit"


In [23]:
# Export
items_df[["item_id", "name", "priority", "preferred_zone", "placement", "placement_note"]].to_csv("ml_placement_recommendations.csv", index=False)
print("✅ ML-based placement recommendations saved to ml_placement_recommendations.csv")

✅ ML-based placement recommendations saved to ml_placement_recommendations.csv
